# 🚌 Projet MDM - Mobilité Durable en Montagne ⛰️

*Auteur : Arnaud Reboud*

*Date : 19/06/2025*

**Description :**

This script loads the List_iti_D4G_isre.csv dataset and extracts the coordinates (X and Y) in EPSG:3857 and converts them to EPSG:4326 (lon, lat). It then add both coordinates to the dataset and saves the results to a CSV file. Original data are conserved.

In [1]:
import geopandas as gpd
import pandas as pd
from pyproj import Transformer

In [2]:
# load dataset
data_raw = pd.read_csv("../data/C2C/Liste_iti_D4G_isere.csv")

# Noms des colonnes sous forme de listes facilement modifiables
list_9_colnames_global_infos = [
    "Titre itinéraire",
    "Id itinéraire",
    "url",
    "activité",
    "durée",
    "dénivelé (m)",
    "longueur (m)",
    "Région/massif",
    "Id wp principal",
]
colname_with_coordinates = "(id WP - titre - [X,Y] - accessibilité (si renseigné) )"

# Multiple rows when there are multiple columns with coordinates
data = (
    data_raw.melt(
        id_vars=list_9_colnames_global_infos,
        value_vars=[col for col in data_raw.columns if "Unnamed" in col].insert(
            0, colname_with_coordinates
        ),
    )
    .dropna(subset=["value"])
    .drop(columns=["variable"])
    .rename(columns={"value": colname_with_coordinates})
    .sort_values(by="Id itinéraire", ascending=False)
)

# Extract X and Y coordinates from the column
X = (
    data[colname_with_coordinates]
    .str.extract(r"(?<= - \[)([^]]+)(?=\])")[0]
    .str.strip()
    .str.split(",", expand=True)[0]
    .astype(float)
)
Y = (
    data[colname_with_coordinates]
    .str.extract(r"(?<= - \[)([^]]+)(?=\])")[0]
    .str.strip()
    .str.split(",", expand=True)[1]
    .astype(float)
)

# Transform coordinates from EPSG:3857 to EPSG:4326
# EPSG:3857 is Web Mercator, EPSG:4326 is WGS84
transformer = Transformer.from_crs(crs_from="EPSG:3857", crs_to="EPSG:4326", always_xy=True)
lon, lat = transformer.transform(X, Y)

# Add coordinates to the DataFrame
data["X"], data["Y"], data["lon"], data["lat"] = X, Y, lon, lat

# Save the DataFrame to a CSV file
data.to_csv("../data/C2C/Liste_iti_D4G_isere_output.csv", index=False)

# create GeoDataFrame with X and Y coordinates in EPSG:3857
gdf_3857 = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(X, Y), crs="EPSG:3857")

# convert GeoDataFrame to EPSG:4326
gdf_4326 = gdf_3857.to_crs("EPSG:4326")